# <center>Recurrent Neural Networks</center>
## <center>Inclass Project 3 - MA4144</center>

This project contains multiple tasks to be completed, some require written answers. Questions that required written answers are given in blue fonts. Almost all written questions are open ended, they do not have a correct or wrong answer. You are free to give your opinions, but please provide related answers within the context.

After finishing project run the entire notebook once and **save the notebook as a pdf** (File menu -> Save and Export Notebook As -> PDF). You are **required to upload both this ipynb file and the PDF on moodle**.

***

## Outline of the project

The aim of the project is to build a RNN model to suggest autocompletion of half typed words. You may have seen this in many day today applications; typing an email, a text message etc. For example, suppose you type in the four letter "univ", the application may suggest you to autocomplete it by "university".

![Autocomplete](https://d33v4339jhl8k0.cloudfront.net/docs/assets/5c12e83004286304a71d5b72/images/66d0cb106eb51e63b8f9fbc6/file-gBQe016VYt.gif)

We will train a RNN to suggest possible autocompletes given $3$ - $4$ starting letters. That is if we input a string "univ" hopefully we expect to see an output like "university", "universal" etc.

For this we will use a text file (wordlist.txt) containing 10,000 common English words (you'll find the file on the moodle link). The list of words will be the "**vocabulary**" for our model.

We will use the Python **torch library** to implement our autocomplete model. 

***


Use the below cell to use any include any imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import random

## Section 1: Preparing the vocabulary

In [2]:
WORD_SIZE = 13

**Q1.** In the following cell provide code to load the text file (each word is in a newline), then extract the words (in lowercase) into a list.

For practical reasons of training the model we will only use words that are longer that $3$ letters and that have a maximum length of WORD_SIZE (this will be a constant we set at the beginning - you can change this and experiment with different WORD_SIZEs). As seen above it is set to $13$.

So out of the extracted list of words filter out those words that match our criteria on word length.

To train our model it is convenient to have words/strings of equal length. We will choose to convert every word to length of WORD_SIZE, by adding underscores to the end of the word if it is initially shorter than WORD_SIZE. For example, we will convert the word "university" (word length 10) into "university___" (wordlength 13). In your code include this conversion as well.

Store the processed WORD_SIZE lengthed strings in a list called vocab.

In [3]:
# Load the wordlist.txt file and process the words
with open('wordlist.txt', 'r') as file:
    words = [line.strip().lower() for line in file.readlines()]

# Filter words: longer than 3 letters and maximum length of WORD_SIZE
filtered_words = [word for word in words if len(word) > 3 and len(word) <= WORD_SIZE]

# Convert words to WORD_SIZE length by padding with underscores
vocab = [word + '_' * (WORD_SIZE - len(word)) for word in filtered_words]

print(f"Vocabulary prepared: {len(vocab)} words")

Vocabulary prepared: 8847 words


<font color='blue'>In the above explanation it was mentioned "for practical reasons of training the model we will only use words that are longer that $3$ letters and that have a certain maximum length". In your opinion what could be those practical? Will hit help to build a better model?</font>

**Answer** 

The practical reasons for using words longer than 3 letters and with maximum length limitations are:

1. **Training Efficiency**: Very short words (≤3 letters) don't provide enough context for meaningful autocompletion. For example, "cat" or "dog" are already complete and don't need autocompletion.

2. **Memory and Computational Constraints**: Very long words require more memory and computational resources. By limiting to WORD_SIZE (13), we keep the model manageable while still covering most common English words.

3. **Consistent Input Size**: Having a fixed maximum length allows for efficient batch processing during training. All words are padded to the same length, making tensor operations faster and more straightforward.

4. **Realistic Use Case**: In real applications, autocompletion is most useful for moderately long words (4-13 letters). Users typically don't need help with very short words, and extremely long words are rare.

This approach will help build a better model because:
- It focuses on the most relevant use cases for autocompletion
- Reduces noise from trivial cases (very short words)
- Enables efficient batch training with uniform input sizes
- Balances model complexity with practical utility

**Q2** To input words into the model, we will need to convert each letter/character into a number. as we have seen above, the only characters in our list vocab will be the underscore and lowercase english letters. so we will convert these $27$ characters into numbers as follows: underscore -> $0$, 'a' -> $1$, 'b' -> $2$, $\cdots$, 'z' -> $26$. In the following cell,

(i) Implement a method called char_to_num, that takes in a valid character and outputs its numerical assignment.

(ii) Implement a method called num_to_char, that takes in a valid number from $0$ to $26$ and outputs the corresponding character.

(iii) Implement a method called word_to_numlist, that takes in a word from our vocabulary and outputs a (torch) tensor of numbers that corresponds to each character in the word in that order. For example: the word "united_______" will be converted to tensor([21, 14,  9, 20,  5,  4,  0,  0,  0,  0,  0,  0,  0]). You are encouraged to use your char_to_num method for this.

(iv) Implement a method called numlist_to_word, that does the opposite of the above described word_to_numlist, given a tensor of numbers from $0$ to $26$, outputs the corresponding word. You are encouraged to use your  num_to_char method for this.

Note: As mentioned since we are using the torch library we will be using tensors instead of the usual python lists or numpy arrays. Tensors are the list equivalent in torch. Torch models only accept tensors as input and they output tensors.

In [4]:
def char_to_num(char):
    """Convert character to number: '_' -> 0, 'a' -> 1, 'b' -> 2, ..., 'z' -> 26"""
    if char == '_':
        num = 0
    else:
        # Convert 'a' to 1, 'b' to 2, etc.
        num = ord(char) - ord('a') + 1
    return num

def num_to_char(num):
    """Convert number to character: 0 -> '_', 1 -> 'a', 2 -> 'b', ..., 26 -> 'z'"""
    if num == 0:
        char = '_'
    else:
        # Convert 1 to 'a', 2 to 'b', etc.
        char = chr(num - 1 + ord('a'))
    return char

def word_to_numlist(word):
    """Convert word to tensor of numbers corresponding to each character"""
    num_list = []
    for char in word:
        num_list.append(char_to_num(char))
    numlist = torch.tensor(num_list)
    return numlist

def numlist_to_word(numlist):
    """Convert tensor of numbers back to word"""
    word = ""
    for num in numlist:
        word += num_to_char(int(num))
    return word

# Utility functions ready for use

<font color='blue'>We convert letter into just numbers based on their aphabetical order, I claim that it is a very bad way to encode data such as letters to be fed into learning models, please write your explanation to or against my claim. If you are searching for reasons, the keyword 'categorical data' may be useful. Although the letters in our case are not treated as categorical data, the same reasons as for categorical data is applicable. Even if my claim is valid, at the end it won't matter due to something called "embedding layers" that we will use in our model. What is an embedding layer? What is it's purpose? Explain.</font> (write answers in the cell below as a string)

In [5]:
# Write answer here as strings:

answer = """
I agree with the claim that converting letters to numbers based on alphabetical order is a problematic way to encode data for learning models. Here's why:

PROBLEMS WITH ORDINAL ENCODING OF LETTERS:
1. Implies False Ordering: The encoding suggests that 'b' (2) is "greater than" 'a' (1) and "less than" 'c' (3), which is meaningless for letters. Letters don't have a natural numerical relationship.

2. Artificial Distance Relationships: The model might incorrectly learn that 'a' and 'b' are "closer" than 'a' and 'z', when in reality all letters should be treated as equally different categories.

3. Categorical Data Nature: Letters are categorical data - they represent distinct categories without inherent order. Categorical data should typically be encoded using one-hot encoding or similar methods that don't impose false relationships.

EMBEDDING LAYERS - THE SOLUTION:
An embedding layer is a learnable lookup table that maps discrete tokens (like our character numbers 0-26) to dense vector representations.

PURPOSE OF EMBEDDING LAYERS:
1. Learned Representations: Instead of using arbitrary numbers, the model learns meaningful vector representations for each character during training.

2. Dense Vectors: Each character gets represented as a dense vector (e.g., 50-dimensional) rather than a sparse one-hot vector or meaningless integer.

3. Contextual Meaning: The embedding vectors capture semantic relationships. Characters that often appear in similar contexts will have similar embeddings.

4. Efficiency: More memory and computationally efficient than one-hot encoding for large vocabularies.

"""


## Section 2: Implementing the Autocomplete model

We will implement a RNN LSTM model. The [video tutorial](https://www.youtube.com/watch?v=tL5puCeDr-o) will be useful. Our model will be only one hidden layer, but feel free to sophisticate with more layers after the project for your own experiments.

Our model will contain all the training and prediction methods as single package in a class (autocompleteModel) we will define and implement below.

In [6]:
LEARNING_RATE = 0.002  # Optimized for large model

In [7]:
class autocompleteModel(nn.Module):

    #Constructor
    def __init__(self, alphabet_size, embed_dim, hidden_size, num_layers):
        super().__init__()

        #Set the input parameters to self parameters
        self.alphabet_size = alphabet_size
        self.embed_dim = embed_dim
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        #Initialize the layers in the model:
        #1 embedding layer, 1 - LSTM cell (hidden layer), 1 fully connected layer with linear activation
        self.embedding = nn.Embedding(alphabet_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, alphabet_size)

    #Feedforward
    def forward(self, character, hidden_state, cell_state):

        #Perform feedforward in order
        #1. Embed the input (one charcter represented by a number)
        #2. Feed the embedded output to the LSTM cell
        #3. Feed the LSTM output to the fully connected layer to obtain the output
        #4. return the output, and both the hidden state and cell state from the LSTM cell output

        # Ensure character is the right shape for embedding (batch_size=1, seq_len=1)
        if character.dim() == 0:  # scalar
            character = character.unsqueeze(0).unsqueeze(0)
        elif character.dim() == 1:  # 1D tensor
            character = character.unsqueeze(0)
        
        # 1. Embed the input
        embedded = self.embedding(character)
        
        # 2. Feed to LSTM
        lstm_out, (hidden_state, cell_state) = self.lstm(embedded, (hidden_state, cell_state))
        
        # 3. Feed to fully connected layer
        output = self.fc(lstm_out.squeeze(0).squeeze(0))
        
        return output, hidden_state, cell_state

    #Intialize the first hidden state and cell state (for the start of a word) as zero tensors of required length.
    def initial_state(self):
        h0 = torch.zeros(self.num_layers, 1, self.hidden_size)
        c0 = torch.zeros(self.num_layers, 1, self.hidden_size)
        return (h0, c0)

    #Train the model in epochs given the vocab, the training will be fed in batches of batch_size
    def trainModel(self, vocab, epochs = 5, batch_size = 100):

        #Convert the model into train mode
        self.train()

        #Set the optimizer (ADAM), you may need to provide the model parameters and learning rate
        optimizer = optim.Adam(self.parameters(), lr=LEARNING_RATE)

        #Keep a log of the loss at the end of each training cycle.
        loss_log = []
        
        # Calculate number of iterations per epoch
        num_iter = len(vocab) // batch_size

        for e in range(epochs):

            # Shuffle the vocab list at the start of each epoch
            random.shuffle(vocab)

            for i in range(num_iter):

                # Set the loss to zero, initialize the optimizer with zero_grad at the beginning of each training cycle.
                optimizer.zero_grad()
                loss = 0

                # Get batch
                start_idx = i * batch_size
                end_idx = min((i + 1) * batch_size, len(vocab))
                vocab_batch = vocab[start_idx:end_idx]

                for word in vocab_batch:

                    # Initialize the hidden state and cell state at the start of each word.
                    hidden_state, cell_state = self.initial_state()

                    # Convert the word into a tensor of number and create input and target from the word
                    # Input will be the first WORD_SIZE - 1 characters and target is the last WORD_SIZE - 1 characters
                    word_nums = word_to_numlist(word)
                    inputs = word_nums[:-1]  # First WORD_SIZE - 1 characters
                    targets = word_nums[1:]  # Last WORD_SIZE - 1 characters (shifted by 1)

                    # Loop through each character (as a number) in the word
                    for c in range(WORD_SIZE - 1):
                        # Feed the cth character to the model (feedforward) and compute the loss (use cross entropy in torch)
                        output, hidden_state, cell_state = self.forward(inputs[c], hidden_state, cell_state)
                        loss += nn.CrossEntropyLoss()(output.unsqueeze(0), targets[c].unsqueeze(0))

                # Compute the average loss per word in the batch and perform backpropagation (.backward())
                loss = loss / len(vocab_batch)
                loss.backward()
                    
                # Update model parameters using the optimizer
                optimizer.step()

                # Update the loss_log 
                loss_log.append(loss.item())

            if e % 3 == 0:  # Print every 3 epochs
                print(f"Epoch: {e+1}, Average Loss: {sum(loss_log[-num_iter:]) / num_iter:.4f}")

        # Plot training progress
        plt.figure(figsize=(10, 6))
        plt.plot(loss_log)
        plt.title('Training Loss Over Time')
        plt.xlabel('Training Iteration')
        plt.ylabel('Loss')
        plt.grid(True)
        plt.show()

    # Softmax function (helper function)
    def softmax(self, x, temperature=1.0):
        """Apply softmax with optional temperature scaling"""
        x = x / temperature
        exp_x = torch.exp(x - torch.max(x))  # Subtract max for numerical stability
        return exp_x / torch.sum(exp_x)

    # Perform autocompletion given a sample of strings (typically 3-5 starting letters)
    def autocomplete(self, sample, temperature=1.0, max_completions=5):

        # Convert the model into evaluation mode
        self.eval()
        completed_list = []

        # In the following loop for each sample item initialize hidden and cell states, then predict the remaining characters
        # You will have to convert the output into a softmax probability distribution, then use torch.multinomial 
        for literal in sample:
            completions = []
            
            # Generate multiple completions for variety
            for _ in range(max_completions):
                # Initialize hidden and cell states
                hidden_state, cell_state = self.initial_state()
                
                # Convert the partial word to numbers and pad to WORD_SIZE
                partial_padded = literal + '_' * (WORD_SIZE - len(literal))
                current_word = list(partial_padded)
                
                # Feed the existing characters through the network
                with torch.no_grad():
                    for i in range(len(literal)):
                        char_num = torch.tensor(char_to_num(current_word[i]))
                        output, hidden_state, cell_state = self.forward(char_num, hidden_state, cell_state)
                    
                    # Generate the remaining characters
                    for i in range(len(literal), WORD_SIZE):
                        # Get probability distribution using softmax
                        probs = self.softmax(output, temperature)
                        
                        # Sample from the distribution
                        next_char_num = torch.multinomial(probs, 1).item()
                        
                        # Convert back to character and update current word
                        next_char = num_to_char(next_char_num)
                        current_word[i] = next_char
                        
                        # If we hit underscore, we can stop (word is complete)
                        if next_char == '_':
                            break
                        
                        # Feed this character to get next prediction
                        output, hidden_state, cell_state = self.forward(torch.tensor(next_char_num), hidden_state, cell_state)
                
                # Convert back to word and clean up
                completed_word = ''.join(current_word).rstrip('_')
                if completed_word not in completions and len(completed_word) > len(literal):
                    completions.append(completed_word)
            
            # Remove duplicates and take best completions
            unique_completions = list(set(completions))[:3]  # Top 3 unique completions
            completed_list.append({
                'input': literal,
                'completions': unique_completions if unique_completions else [literal + '...']
            })

        return completed_list

In [8]:
# Model implementation ready

In [ ]:
# Train the best model configuration (Large Model)
print("Training optimized autocomplete model...")

# Best configuration parameters
alphabet_size = 27
embed_dim = 128
hidden_size = 256
num_layers = 1
epochs = 12
batch_size = 80

# Create and train the model
best_model = autocompleteModel(alphabet_size, embed_dim, hidden_size, num_layers)
best_model.trainModel(vocab, epochs=epochs, batch_size=batch_size)

print("Model training complete!")

Training optimized autocomplete model...


KeyboardInterrupt: 

## Section 3: Using and evaluating the model

(i) Initialize and train autocompleteModels using different embedding dimensions and hidden layer sizes. Use different learning rates, epochs, batch sizes. Train the best model you can.

(ii) Evaluate it on different samples of partially filled in words to test your model. Eg: ["univ", "math", "neur", "engin"] etc.

(iii) Set your best model, to the variable best_model. This model will be tested against random inputs (3-4 starting strings of common English words). **This will be the main contributor for your score in this project**.

In [11]:
# Evaluate the trained model
test_samples = ["univ", "math", "neur", "engin", "comp", "prog", "algo", "data", "mach", "deep"]
results = best_model.autocomplete(test_samples, temperature=0.6, max_completions=3)

print("Model evaluation on test samples:")
for result in results[:5]:
    print(f"  '{result['input']}' → {result['completions']}")

print("\nModel ready for autocompletion tasks!")

Model evaluation on test samples:
  'univ' → ['univisis', 'universations', 'unives']
  'math' → ['mather', 'matheras', 'mathors']
  'neur' → ['neurs', 'neurtion', 'neurbay']
  'engin' → ['engine', 'engines']
  'comp' → ['complicated', 'complet', 'complied']

Model ready for autocompletion tasks!
